In [43]:
#Dependencies
import numpy as np
import pandas as pd
import datetime as dt
import re

In [2]:
os.getcwd()

'/Users/jacosta3/OneDrive - University of South Florida/Gerdau Projects/pft/scripts'

## Fetching Indicators info

In [89]:
#File directory
ser_path = "./serialized/"
path = "./confidential/"
#change directory to datasets
os.chdir("../datasets")

In [90]:
indicators = pd.read_excel(path+"market_indicators.xlsx",sheet_name="Monthly")

In [91]:
indicators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 17 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   dd/mm/yy                                             124 non-null    datetime64[ns]
 1   Year                                                 124 non-null    int64         
 2   Month                                                124 non-null    object        
 3   National Architect Billing/monthly                   121 non-null    float64       
 4   Total Federal Construction/Billion $                 121 non-null    float64       
 5   Total State and Local Construction/Billion $         121 non-null    float64       
 6   US Steel Capacity Utilization                        109 non-null    float64       
 7   US Long Steel/million $ per short ton                26 non-null     float64       
 8   

## Cleaning Columns

In [92]:
def clean_columns(dataframe):
    for col in dataframe.columns:
        dataframe.rename(columns={col:re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', col)}, inplace=True)
    dataframe.columns = dataframe.columns.str.strip().str.lower().str.replace(" ","_")
    try:
        dataframe["calendar_day"] = dataframe["calendar_day"].astype("datetime64")
    except:
        pass
    try:
        dataframe["requested_date"] = dataframe["requested_date"].astype("datetime64")
        dataframe["confirmed_date"] = dataframe["confirmed_date"].astype("datetime64")
        dataframe["material_avail_date"] = dataframe["material_avail_date"].astype("datetime64")
        dataframe["load_date"] = dataframe["load_date"].astype("datetime64")
        dataframe["plan_goods_issue_date"] = dataframe["plan_goods_issue_date"].astype("datetime64")
    except:
        pass
    try:
        dataframe["snapshot_date"] = dataframe["snapshot_date"].astype("datetime64")
    finally:
        return dataframe.columns

In [93]:
clean_columns(indicators)

Index(['dd/mm/yy', 'year', 'month', 'national_architect_billing/monthly',
       'total_federal_construction/billion_$',
       'total_state_and_local_construction/billion_$',
       'us_steel_capacity_utilization',
       'us_long_steel/million_$_per_short_ton',
       'us_domestic_hot-rolled_coil/million_$_per_short_ton',
       'msci_carbon_bar_monthly_shipments',
       'msci_carbon_bar_monthly_shipments_(canada)',
       'structural_shipments_/usa', 'structural_shipments_/canada',
       'ism_manufacturing', 'ism_non_manufacturing',
       'global_steel_utilization', 'chicago#1_scrap_bushelling'],
      dtype='object')

### Create Month & Year Columns

In [94]:
indicators['year'] = indicators['dd/mm/yy'].dt.year
indicators['month'] = indicators['dd/mm/yy'].dt.month

### Rename Columns

In [96]:
indicators.columns

Index(['dd/mm/yy', 'year', 'month', 'national_architect_billing/monthly',
       'total_federal_construction/billion_$',
       'total_state_and_local_construction/billion_$',
       'us_steel_capacity_utilization',
       'us_long_steel/million_$_per_short_ton',
       'us_domestic_hot-rolled_coil/million_$_per_short_ton',
       'msci_carbon_bar_monthly_shipments',
       'msci_carbon_bar_monthly_shipments_(canada)',
       'structural_shipments_/usa', 'structural_shipments_/canada',
       'ism_manufacturing', 'ism_non_manufacturing',
       'global_steel_utilization', 'chicago#1_scrap_bushelling'],
      dtype='object')

In [98]:
#Remane column & pick only desired ones 
indicators = indicators.rename(columns={'dd/mm/yy':'calendar_day',
                                        'total_federal_construction/billion_$':'fed_construction$B',                                                              'total_state_and_local_construction/billion_$':'state_local_construction$B',
                                        'msci_carbon_bar_monthly_shipments':'carbon_bar_ship_usa',
                                        'msci_carbon_bar_monthly_shipments_(canada)':'carbon_bar_ship_canada',
                                        'structural_shipments_/usa':'structural_ship_usa', 
                                        'structural_shipments_/canada':'structural_ship_canada',
                                        'chicago#1_scrap_bushelling':'chicago_scrap$ST'})
indicators = indicators[['calendar_day','year','month', 'global_steel_utilization','fed_construction$B',                                           'state_local_construction$B','carbon_bar_ship_usa','carbon_bar_ship_canada',                                              'structural_ship_usa','structural_ship_canada', 'ism_manufacturing', 'ism_non_manufacturing',                             'chicago_scrap$ST']]

### Selecting 2016 dates on

In [99]:
#select 2016-2020
indicators = indicators[(indicators['calendar_day'].dt.year >= 2016) & (indicators['calendar_day'] <='2020-01-01')]
indicators = indicators.reset_index()

In [100]:
indicators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   index                       49 non-null     int64         
 1   calendar_day                49 non-null     datetime64[ns]
 2   year                        49 non-null     int64         
 3   month                       49 non-null     int64         
 4   global_steel_utilization    45 non-null     float64       
 5   fed_construction$B          49 non-null     float64       
 6   state_local_construction$B  49 non-null     float64       
 7   carbon_bar_ship_usa         49 non-null     float64       
 8   carbon_bar_ship_canada      49 non-null     float64       
 9   structural_ship_usa         49 non-null     float64       
 10  structural_ship_canada      49 non-null     float64       
 11  ism_manufacturing           49 non-null     float64       
 

### Fill NaNs

In [101]:
#filling NaNs - assumption gradual increase 
values = {36:0.7768,37:0.7915,38:0.8063,39:0.8210}
indicators['global_steel_utilization'] = indicators['global_steel_utilization'].fillna(value=values, axis=0)

In [102]:
indicators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   index                       49 non-null     int64         
 1   calendar_day                49 non-null     datetime64[ns]
 2   year                        49 non-null     int64         
 3   month                       49 non-null     int64         
 4   global_steel_utilization    49 non-null     float64       
 5   fed_construction$B          49 non-null     float64       
 6   state_local_construction$B  49 non-null     float64       
 7   carbon_bar_ship_usa         49 non-null     float64       
 8   carbon_bar_ship_canada      49 non-null     float64       
 9   structural_ship_usa         49 non-null     float64       
 10  structural_ship_canada      49 non-null     float64       
 11  ism_manufacturing           49 non-null     float64       
 

## Save to file

In [104]:
#indicators.to_pickle(ser_path+'indicators.pkl')

In [ ]:
### Ready for merging 
# Consider whether monthly value ingestion may be better than weekly